In [5]:
from KnapsackNetsModified import SafetyNet
#from KnapsackNets import SafetyNet
import torch
from torch.autograd import Variable
from torch.autograd import Function
from torch.nn import Parameter


# Generate Simulation Parameters

From JK thesis!
- https://www.cmu.edu/tepper/programs/phd/program/assets/dissertations/2017-joint-phd-program-algorithms-combinatorics-and-optimization-karp-dissertation.pdf

```
Note that at larger cluster numbers, the improved performance of our
knapsack models will in fact be significantly more pronounced but even as few as
50 clusters is enough to make a significant impact. Demand for each cluster is mod-
eled as a Poisson process whose rate parameter is drawn from a Normal distribution
with mean 10 and standard deviation 5. Each day of the simulation allows the Pois-
son processes generating demand to run for either 40 or 400 time units (depending
on simulation condition). Inventory distributions for each category are modeled as
Poisson distributions with rate parameters drawn from a Uniform distribution from
1 to 20. The price of items in each category is drawn from a Normal distribution
with mean 800 and standard deviation 400. Each order is assigned a cancel probabil-
ity based on a logistic function of the form 1 − 1/1+ea+b·I , where I is the item’s stated 
inventory level and a and b are parameters of the item’s category. a and b are drawn for each category from Normal distribution with mean .5 and −.2 and standard deviations .1 and either .05 or .15 (depending on simulation condition), respectively. The inventory distribution parameters, cancel parameters, demand rates, and prices are all truncated at 0 to prevent negative values.


We run simulations of 8 scenarios that vary across the following conditions:
amount of data truncation, volume of demand, and variability of cancel rate. We
test two parameter options for each of these dimensions of variation. We vary data
truncation by setting our benchmark Retail-1-threshold policies to single thresholds
of either 8 or 14, allowing approximately 1/3 and 2/3 of total demand to get truncated,
respectively. Volume of demand is varied by allowing either 40 or 400 time units
for the demand generating Poisson process during each period of the simulation.
We control cancel variability by running simulations where the standard deviation
of cancel parameter b is .05 as well as .15. We test the Onera and Retail-1-threshold
policies on 10 runs of each of the 8 (2/3) combinations of these simulation variants.
We wish to compare against corresponding Onera policies. We use the corresponding
Retail-1-threshold policy as the starting threshold for the Onera policies and then
let the Onera policies train on the data it collects and determine its own thresholds
for the remaining days. At the end of the simulation, we compare the Onera policy
to the Retail-1-threshold policy in terms of cancel rate and revenues.
```

In [32]:
import numpy.random as npr
import numpy as np
import random 

global_seed = 0
random.seed(global_seed)
npr.seed(global_seed)

class SafetyNetSimulation():
    def __init__(self, 
                 is_high_demand=False, 
                 is_high_truncation=False, 
                 is_high_variable_cancel_rate=False,
                 nCategories = 50, nThresholds = 20, 
                 n_demand_mean = 10, n_demand_stddev = 5,
                 n_inventory_rate_min = 1, n_inventory_rate_max = 20, # inventory rate bounds
                 category_price_mean = 800, category_price_stddev = 400, # price moments
                 a_mean = .5, a_stddev = .1, b_mean = -.2): # cancel params
        
        self.is_high_demand = is_high_demand
        self.is_high_truncation = is_high_truncation
        self.is_high_variable_cancel_rate = is_high_variable_cancel_rate
        
        self.nCategories = nCategories
        self.nThresholds = nThresholds
         
        """
        Each day of the simulation allows the Poisson processes generating 
        demand to run for either 40 or 400 time units (depending on simulation condition)
        """
        if self.is_high_demand:
            self.n_steps = 400
        else:
            self.n_steps = 40
            
        """
        We vary data truncation by setting our benchmark Retail-1-threshold policies to single thresholds
        of either 8 or 14, allowing approximately 1/3 and 2/3 of total demand to get truncated,
        respectively.
        """
        if self.is_high_truncation:
            self.retail_1_threshold_policy = 14
        else:
            self.retail_1_threshold_policy = 8
        
        """
        We control cancel variability by running simulations where the standard deviation
        of cancel parameter b is .05 as well as .15.
        """
        if self.is_high_variable_cancel_rate:
            self.b_stddev = .15
        else:
            self.b_stddev = .05
            

        # Lambdas for Poisson Process generating Demand
        self.category_demand_rate = npr.normal(loc=n_demand_mean, 
                                               scale=n_demand_stddev, 
                                               size=self.nCategories)
        
        self.category_demand_rate = np.clip(self.category_demand_rate, a_min=0, a_max=None)

        # AVG PRICE PER CATEGORY
        self.category_price_mean = category_price_mean
        self.category_price_stddev = category_price_stddev

        # INVENTORY LEVEL
        self.category_inventory_level_rate = npr.uniform(high=n_inventory_rate_max, 
                                                    low=n_inventory_rate_min, 
                                                    size=n_category_clusters)
        
        self.category_inventory_level_rate = np.clip(self.category_inventory_level_rate, 
                                                     a_min=0, a_max=None)

        # CANCELLATION PROBABILITY
        self.a_param = npr.normal(loc=a_mean, scale=a_stddev, size=n_category_clusters)
        self.b_param = npr.normal(loc=b_mean, scale=b_stddev, size=n_category_clusters)
        
    def category_demand_to_orders(self, category_demand):
        """
        1 hot encode a category demand (nCategories,) as a matrix with dim (nOrders, nCategories)
        Returned matrix tells us the category for each order
        """
        rows = []
        lc = len(category_demand)
        for i, a in enumerate(category_demand):
            for _ in range(a):
                z = np.zeros(lc)
                z[i] = 1
                rows.append(z)
        if len(rows) > 0:
            return np.stack(rows)
        else:
            return np.array(rows)

    def inv_count_from_orders(self, orders):
        """
        For each order in (nOrders, nCategories), generates an inventory level using a poisson process 
        and 1-hot encodes the inventory at the time of the order in a (nOrders, nThresholds) matrix
        If inventory exceeds nThresholds, cap it at nThreshold 
        [Not sure if this is the proper way to handle it]
        """
        inv_count = []
        for o in orders:
            category_ind = np.argmax(o)
            inv_count_row = np.zeros(shape=(self.nThresholds,))
            inv_level = npr.poisson(self.category_inventory_level_rate[category_ind])
            if inv_level >= self.nThresholds:
                inv_level = self.nThresholds-1
            inv_count_row[inv_level] = 1
            inv_count.append(inv_count_row)

        return np.array(inv_count)

    def price_from_orders(self, orders):
        """
        For each order, generate a price. 
        Returns a (nOrders,) vector
        """
        return npr.normal(loc=self.category_price_mean, 
                          scale=self.category_price_stddev, 
                          size=len(orders))

    def cancel_prob(self, orders, inv_count):
        """
        For each order, calculate a cancellation probability according to that categories params
        returns a (nOrders,) vector
        """
        order_cat = np.argmax(orders, axis= 1) # get category for each order
        inv_cat = np.argmax(inv_count, axis=1) # get inventory for each order
        a_param_all = [self.a_param[i] for i in order_cat]
        b_param_all = [self.b_param[i] for i in order_cat]
        cancel_probs = 1.0 - 1.0/(1.0+ np.exp(a_param_all + (b_param_all * inv_cat)))
        return cancel_probs
    
    def generate_data(self):
        
        X_T = [np.random.poisson(np.clip(rate, a_min=0, a_max=None) , size=self.n_steps) 
               for rate in self.category_demand_rate]
        self.category_demand = np.array([np.sum(X) for X in X_T])
             
        orders = self.category_demand_to_orders(category_demand)
        np.random.shuffle(orders)

        inv_count = self.inv_count_from_orders(orders)
        price = self.price_from_orders(orders)
        cancel_probs = self.cancel_prob(orders, inv_count)

        collection_thresholds = np.zeros(shape=(orders.shape[0],self.nThresholds))
        # do we subtract 1 or not?
        collection_thresholds[:,self.retail_1_threshold_policy-1]=1
        
        # Some sanity checks
        assert orders.shape == (orders.shape[0], self.nCategories)
        assert inv_count.shape == (orders.shape[0], self.nThresholds)
        assert price.shape == (orders.shape[0],)
        assert cancel_probs.shape == (orders.shape[0],)
        assert collection_thresholds.shape == (orders.shape[0],self.nThresholds)
        
        
        return orders, inv_count, price, cancel_probs, collection_thresholds
    

            
        


In [33]:
safety_net_simulation = SafetyNetSimulation(
                 is_high_demand=False, 
                 is_high_truncation=False, 
                 is_high_variable_cancel_rate=False)

In [34]:
orders, inv_count, price, cancel_probs, collection_thresholds = safety_net_simulation.generate_data()

In [35]:
# First Pass, turn off LP layer
safety_net = SafetyNet(
    nKnapsackCategories=safety_net_simulation.nCategories, 
    nThresholds=safety_net_simulation.nThresholds, 
    starting_thresholds=torch.ones(safety_net_simulation.nCategories, safety_net_simulation.nThresholds), # all possible thresholds
    parametric_knapsack=False # end to end safety net if true, no optimization lp layer if False 
    #(net will directly update the weights)
)

(new_revenue_loss, 
 new_cancel_constraint_loss, 
 new_accept_constraint_loss, 
 arrival_probability_batch_by_threshold, 
 log_arrival_prob, 
 log_cancel_prob, 
 log_category_prob, 
 estimated_batch_total_demand, 
 observed_cancel_constraint_loss, 
 observed_accept_constraint_loss, 
 lp_infeasible) = safety_net.forward(
    category=Variable(torch.from_numpy(orders.astype(np.float32))),
    inv_count=torch.from_numpy(inv_count.astype(np.float32)),
    price=torch.from_numpy(price.astype(np.float32)),
    cancel=torch.from_numpy(cancel_probs.astype(np.float32)),
    collection_thresholds=torch.from_numpy(collection_thresholds.astype(np.float32))
)



In [ ]:

# category clusters
n_category_clusters = 50

# distribution of rate parameter for poisson process that generates demand for each cluster
n_demand_mean = 10
n_demand_stddev = 5

# Number of steps to run the poisson process for
n_steps = 40 # or 400

# rate paramter for possion distribution that models inventory for each category
n_inventory_rate_min = 1
n_inventory_rate_max = 20

# moments for normal distribution characterizing price
category_price_mean = 800
category_price_stddev = 400

# Simulations params for cancellation
a_mean = .5
a_stddev = .1

b_mean = -.2
b_stddev = .05

# DEMAND 
category_demand_rate = npr.normal(loc=n_demand_mean, scale=n_demand_stddev, size=n_category_clusters)
X_T = [np.random.poisson(np.clip(rate, a_min=0, a_max=None) , size=n_steps) for rate in category_demand_rate]
category_demand = np.array([np.sum(X) for X in X_T])

# AVG PRICE PER CATEGORY
category_price = npr.normal(loc=category_price_mean, scale=category_price_stddev, size=n_category_clusters)

# INVENTORY LEVEL
category_inventory_level_rate = npr.uniform(high=n_inventory_rate_max, 
                                            low=n_inventory_rate_min, 
                                            size=n_category_clusters)
category_inventory_level = npr.poisson(lam=category_inventory_level_rate)

# CANCELLATION PROBABILITY
a_param = npr.normal(loc=a_mean, scale=a_stddev, size=n_category_clusters)
b_param = npr.normal(loc=b_mean, scale=b_stddev, size=n_category_clusters)

category_cancel_prob = 1.0 - 1.0/(1.0+ np.exp(a_param + (b_param * category_inventory_level)))

# Store and Clip
params = {
    'inventory': category_inventory_level,
    'demand': category_demand,
    'price': category_price,
    'cancellation_probability': category_cancel_prob
}

for k,v in params.items():
    params[k] = np.clip(v, a_min=0, a_max=None)

    
# safety net params
nCategories = 50
nThresholds = 20

In [7]:
def category_demand_to_orders(category_demand):
    rows = []
    lc = len(category_demand)
    for i, a in enumerate(category_demand):
        for _ in range(a):
            z = np.zeros(lc)
            z[i] = 1
            rows.append(z)
    if len(rows) > 0:
        return np.stack(rows)
    else:
        return np.array(rows)

def inv_count_from_orders(category_inventory_level_rate, orders):
    inventory = []
    for o in orders:
        category_ind = np.argmax(o)
        inv_count_row = np.zeros(shape=(nThresholds,))
        inv_level = npr.poisson(category_inventory_level_rate[category_ind])
        if inv_level >= nThresholds:
            inv_level = nThresholds-1
        inv_count_row[inv_level] = 1
        inventory.append(inv_count_row)
    
    return np.array(inventory)

def price_from_orders(category_price_mean,category_price_stddev, orders):
    return npr.normal(loc=category_price_mean, scale=category_price_stddev, size=len(orders))

def cancel_prob(a_param, b_param, orders, inv_count):
    order_cat = np.argmax(orders, axis= 1)
    inv_cat = np.argmax(inv_count, axis=1)
    a_param_all = [a_param[i] for i in order_cat]
    b_param_all = [b_param[i] for i in order_cat]
    cancel_probs = 1.0 - 1.0/(1.0+ np.exp(a_param_all + (b_param_all * inv_cat)))
    return cancel_probs
    

orders = category_demand_to_orders(category_demand)
np.random.shuffle(orders)

inv_count = inv_count_from_orders(category_inventory_level_rate, orders)
price = price_from_orders(category_price_mean, category_price_stddev, orders)
cancel_probs = cancel_prob(a_param, b_param, orders, inv_count)

start_thresh = 8
collection_thresholds = np.zeros(shape=(orders.shape[0],nThresholds))
collection_thresholds[:,start_thresh-1]=1

# 1st Pass

In [14]:
temp_model = SafetyNet(
    nKnapsackCategories=nCategories, 
    nThresholds=nThresholds, 
    starting_thresholds=torch.ones(nCategories, nThresholds), # all possible thresholds
    parametric_knapsack=False # end to end safety net if true, no optimization lp layer if False 
    #(net will directly update the weights)
)

(new_revenue_loss, 
 new_cancel_constraint_loss, 
 new_accept_constraint_loss, 
 arrival_probability_batch_by_threshold, 
 log_arrival_prob, 
 log_cancel_prob, 
 log_category_prob, 
 estimated_batch_total_demand, 
 observed_cancel_constraint_loss, 
 observed_accept_constraint_loss, 
 lp_infeasible) = temp_model.forward(
    category=Variable(torch.from_numpy(orders.astype(np.float32))),
    inv_count=torch.from_numpy(inv_count.astype(np.float32)),
    price=torch.from_numpy(price.astype(np.float32)),
    cancel=torch.from_numpy(cancel_probs.astype(np.float32)),
    collection_thresholds=torch.from_numpy(collection_thresholds.astype(np.float32))
)



In [15]:
new_objective_loss

tensor(-1289.6364)

Training 




- split our orders into batches, forward/backward for each batch of period 1 data
- difference between the *new_* and *observered_*
- where does likelihood measures fit into return values from forward pass
- define the logic for updates -> but how are we measuring if a violation has occured?
- what are the init parameters?
- how do we update the target cancellation rate based on period 1 cancellation rate


In [12]:
res

(tensor(-1289.6364),
 tensor(43568.9648),
 tensor(2476.2925),
 tensor([[ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07],
         [ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07],
         [ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07],
         ...,
         [ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07],
         [ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07],
         [ 1.4858e-08,  4.4574e-08,  6.6861e-08,  ...,  8.3788e-07,
           3.8834e-07,  3.1263e-07]]),
 tensor([[-17.5101, -16.7237, -16.3813,  ..., -13.9805, -14.7360,
          -14.9468],
         [-17.5101, -16.7237, -16.3813,  ..., -13.9805, -14.7360,
          -14.9468],
         [-17.5101, -16.7237, -16.3813,  ..., -13.9805, -14.7360,
          -14.9468],
         ...,
         [-17.510

# Training after 1st pass

In [ ]:
"""
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

"""


In [ ]:
temp_model.parametric_knapsack = True

temp_model.forward(
    category=Variable(torch.from_numpy(orders.astype(np.float32))),
    inv_count=torch.from_numpy(inv_count.astype(np.float32)),
    price=torch.from_numpy(price.astype(np.float32)),
    cancel=torch.from_numpy(cancel_probs.astype(np.float32)),
    collection_thresholds=torch.from_numpy(collection_thresholds.astype(np.float32))
)

# Initialize Safety Net

In [ ]:
temp_model = SafetyNet(
    nKnapsackCategories=nCategories, 
    nThresholds=nThresholds, 
    starting_thresholds=torch.ones(nCategories, nThresholds), # all possible thresholds
    parametric_knapsack=True # end to end safety net if true, no optimization lp layer if False 
    #(net will directly update the weights)
)


# Intialize Optimization/Estimation 

In [6]:
# JK provided learning rates
temp_optimizer_est = torch.optim.SGD([
{'params':[temp_model.prices_est]}, #what will happen here since there is no learning rate? will it always stay the same?
{'params':[temp_model.inventory_lam_est], 'lr': 1e-0},
{'params':[temp_model.demand_distribution_est], 'lr': 1e-2},
{'params':[temp_model.cancel_coef_est], 'lr': 5e-3},
{'params':[temp_model.cancel_intercept_est], 'lr': 1e-1}
    ], lr=1e-7)

temp_optimizer_opt = torch.optim.SGD([
{'params':[temp_model.inventory_lam_opt], 'lr': 2e-5},
{'params':[temp_model.demand_distribution_opt], 'lr': 1e-5},
{'params':[temp_model.cancel_coef_opt], 'lr': 2e-7},
{'params':[temp_model.cancel_intercept_opt], 'lr': 2e-6}
    ], lr=1e-8)

temp_optimizer_RHS =torch.optim.SGD([
{'params':[temp_model.cancel_rate_param, temp_model.accept_rate_param], 'lr': 1e-6}
    ], lr=1e-8)

In [ ]:
## Period 1 training

# zero grad
temp_optimizer_est.zero_grad()
temp_optimizer_opt.zero_grad()
temp_optimizer_RHS.zero_grad()

# backward passes
if is_cancel_constraint_violated:
    new_cancel_constraint_loss.backward()
    temp_optimizer_RHS.step()
elif is_other_constraints_violated:
    new_accept_constraint_loss.backward()
    temp_optimizer_RHS.step()
else:
    new_revenue_loss.backward()
    temp_optimizer_est.step()
    # manually copy values of temp_optimizer_est into temp_optimizer_opt params 

## From period 2 onwards

# zero grad
temp_optimizer_est.zero_grad()
temp_optimizer_opt.zero_grad()
temp_optimizer_RHS.zero_grad()

# backward
if is_cancel_constraint_violated:
    new_cancel_constraint_loss.backward()
    temp_optimizer_RHS.step()
elif is_other_constraints_violated:
    new_accept_constraint_loss.backward()
    temp_optimizer_RHS.step()
else:
    new_revenue_loss.backward()
    temp_optimizer_opt.step()
    temp_optimizer_est.step()
    
    

## Forward Pass 

Number of orders == Batch Size or nBatch

Category: what is the category for each order -> (nBatch x nCategory)

Inv Count: what is the inventory count for each order? nBatch x 1 (makes sense). nBatch x nThresholds works. ??
- we may be using a cracked version from changes we made. try to rerun with a clean version of KnapsackNets.py

Price: what is the price for each order? nBatch x 1

Cancel: What is the cancel status (probability or binary?) for each order? 

Collection Thresholds: ??? dimensions indicate all possible thresholds for all orders.



In [ ]:
nBatch = 32
temp_model.forward(
    category=Variable(torch.from_numpy(category_demand.reshape(1,nCategories).astype(np.float32)).expand(nBatch,nCategories)),
    inv_count=torch.from_numpy(category_inventory_level[:nThresholds].reshape(1,nThresholds).astype(np.float32)).expand(nBatch,nThresholds),
    price=torch.from_numpy(category_price[:nBatch].reshape(nBatch).astype(np.float32)).expand(nBatch),
    cancel=torch.from_numpy(category_cancel_prob[:nBatch].reshape(nBatch).astype(np.float32)).expand(nBatch),
    collection_thresholds=torch.ones(nBatch,nThresholds)*10
)


# Exploring Poisson Function


In [ ]:

class PoissonFunction(Function):
    def __init__(self,nKnapsackCategories,nThresholds, verbose=False):
        self.verbose = verbose
        self.nKnapsackCategories = nKnapsackCategories
        self.nThresholds = nThresholds
    def forward(self, lam, thresholds):
        self.poisson_dist = None
        self.lam_matrix = torch.ger(lam,torch.ones(self.nThresholds)) # C X T -> 3's
        self.t_matrix = torch.ger(torch.ones(self.nKnapsackCategories),thresholds) # C x T -> 0's
        self.lam_pow_t_div_t_factorial = LamToverTFactorial(self.nKnapsackCategories,self.nThresholds,thresholds,lam)
        #lam_pow_t_matrix = torch.pow(self.lam_matrix,self.t_matrix)
        self.exp_lam_matrix = torch.exp(-1*self.lam_matrix)
        #t_factorial = FactorialFunctionPlain(self.nThresholds,thresholds)
        #self.t_factorial_matrix = torch.ger(torch.ones(self.nKnapsackCategories),t_factorial)
        #print("PoissonFunction t_factorial_matrix",self.t_factorial_matrix)
        self.poisson_dist = self.lam_pow_t_div_t_factorial*self.exp_lam_matrix
        #print("PoissonFunction lam_pow_t_div_t_factorial",self.lam_pow_t_div_t_factorial)
        #print("PoissonFunction exp_lam_matrix",self.exp_lam_matrix)
        #print("PoissonFunction poisson_dist",self.poisson_dist)
        self.save_for_backward(lam)
        return self.poisson_dist
    def backward(self, grad_output):
        #print("PoissonFunction grad_output", grad_output)
        grad_lam = grad_thresholds = None
        #lam_t_minus_one_matrix = torch.pow(self.lam_matrix,self.t_matrix-1)
        lam_pow_t_minus_one_div_t_factorial = torch.div(self.lam_pow_t_div_t_factorial,self.lam_matrix)
        t_minus_lam_matrix = self.t_matrix-self.lam_matrix
        #print("PoissonFunction exp_lam_matrix",self.exp_lam_matrix)
        #print("PoissonFunction lam_pow_t_minus_one_div_t_factorial", lam_pow_t_minus_one_div_t_factorial)
        dp_dlam = self.exp_lam_matrix*lam_pow_t_minus_one_div_t_factorial*t_minus_lam_matrix
        #print("PoissonFunction dp_dlam", dp_dlam)
        grad_lam = torch.sum(grad_output*dp_dlam,dim=1).squeeze()
        #print("PoissonFunction grad_lam", grad_lam)
        return grad_lam, grad_thresholds

def LamToverTFactorial(nKnapsackCategories,nThresholds,t_vector,lam_vector):
    soln_matrix = torch.ones(nKnapsackCategories,nThresholds)
    lam_matrix = torch.ger(lam_vector, torch.ones(nThresholds))
    onesCategories = torch.ones(nKnapsackCategories)
    t_matrix = torch.ger(torch.ones(nKnapsackCategories),t_vector)
    for i in range(nThresholds):
        lam_matrix = lam_matrix.t()
        lam_matrix[i]=onesCategories
        lam_matrix = lam_matrix.t()
        soln_matrix = torch.div(soln_matrix*lam_matrix,(t_matrix-i).clamp(min=1))
    return soln_matrix

In [ ]:

pf = PoissonFunction(nCategories,nThresholds,verbose=-1)
eps=1e-8
thresholds = Variable(torch.arange(0,nThresholds))
inventory_initializer = 3
inventory_lam_est = Parameter(torch.ones(nCategories)*inventory_initializer)

In [ ]:
inventory_distribution_raw_est = pf(inventory_lam_est,thresholds)+ eps

In [ ]:
inventory_distribution_raw_est.min()

# Cancel Rate by Category

In [ ]:
def cancel_rate_belief_cXt(cancel_coefs,cancel_intercepts,thresholds_matrix):
    #1-(1/(1+math.exp(cancel_intercept+x*cancel_coef)
    cancel_intercepts_matrix = cancel_intercepts.unsqueeze(1).expand_as(thresholds_matrix)
    cancel_coefs_matrix = cancel_coefs.unsqueeze(1).expand_as(thresholds_matrix)
    intercept_plus_thresh_times_coef = cancel_intercepts_matrix+(thresholds_matrix*cancel_coefs_matrix)
    exp_matrix = torch.exp(intercept_plus_thresh_times_coef)
    cancel_rate_belief = 1-(1/(1+exp_matrix))
    return cancel_rate_belief

In [ ]:
'''
self.belief_cancel_rate_cXt_est = cancel_rate_belief_cXt(self.cancel_coef_neg_est,
                                                         self.cancel_intercept_est,
                                                         self.thresholds.unsqueeze(0).expand(self.nKnapsackCategories,self.nThresholds))
'''

In [ ]:
torch.sum(torch.ones(4,4))